# Introduction
This notebook extends our initial project on translating sign language. The focus here is on assessing the robustness of our model against potential sensor malfunctions. We simulate malfunctions by intentionally altering sensor data and evaluate the model's performance across various sensor combinations. This exercise aims to understand how sensor failures might impact the accuracy and reliability of our model.


# Review of Initial Model Development
This section revisits the steps taken in our previous notebook, including data preprocessing, model building, and initial training. These foundational steps provide the context for our current focus on evaluating model robustness.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
import joblib
import matplotlib.pyplot as plt
from keras.layers import SimpleRNN, Bidirectional, BatchNormalization
from sklearn.metrics import precision_score, recall_score, f1_score

df_1 = pd.read_csv('../dataset/sensor_data_badr.csv')
df_2 = pd.read_csv('../dataset/sensor_data_mouad.csv')
df_3 = pd.read_csv('../dataset/sensor_data_ismail.csv')
# Concatenate the three dataframes
df = pd.concat([df_1, df_2, df_3], ignore_index=True)

# number of rows and columns
print(df.shape)

# Convert all feature columns to numeric and set non-convertible values to NaN
for col in df.columns[:-1]:  # Excluding the last column
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Removing rows with NaN values
df.dropna(inplace=True)

# Separate features and labels
X = df.iloc[:, :-1].values  # All columns except the last one
y = df.iloc[:, -1].values   # Only the last column

# Scale the features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Reshape X to fit the RNN model (samples, time steps, features)
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Encode the labels
encoder = OneHotEncoder(sparse=False)
y_encoded = encoder.fit_transform(y.reshape(-1, 1))

# Define the RNN model
model_rnn = Sequential()
model_rnn.add(Bidirectional(SimpleRNN(30, activation='relu', return_sequences=True), input_shape=(X.shape[1], X.shape[2])))
model_rnn.add(BatchNormalization())
model_rnn.add(SimpleRNN(32, activation='relu'))
model_rnn.add(Dropout(0.3))
model_rnn.add(Dense(16, activation='relu'))
model_rnn.add(Dense(y_encoded.shape[1], activation='softmax'))

# Compile the model with categorical_crossentropy loss function
model_rnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Add EarlyStopping as a callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model_rnn.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Save the model
model_rnn.save('rnn_model.h5')
# Save the scaler to use it in predict.py and scale the realtime data
joblib.dump(scaler, 'rnn_scaler.joblib')

# Sensor Malfunction Simulation
We simulate sensor malfunctions by intentionally setting the data for specific sensors to zero. This approach helps us to understand how the model performs when certain sensors fail or provide incorrect readings, a scenario that could occur in real-world usage.

## Useful functions

In [41]:
def malfunction(X, sensor, value):
    if sensor.startswith('Position') or sensor.startswith('Orientation'):
        for col in df.columns:
            # Get the column starting with the sensor name and has the hand name in it (left or right)
            hand = sensor.split('-')[1]
            if col.startswith(sensor.split('-')[0]) and hand in col:
                X[:, :, df.columns.get_loc(col)] = value
    else:
        for col in df.columns:
            if col.startswith(sensor):
                X[:, :, df.columns.get_loc(col)] = value

In [42]:
def predict(X):
    y_pred = model_rnn.predict(X)
    # Convert predictions to classes
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)
    # Calculate the accuracy
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    print(f"Accuracy on the test set: {accuracy * 100:.2f}%")

    # Calculate precision, recall, and F1-score
    precision = precision_score(y_test_classes, y_pred_classes, average='weighted')
    recall = recall_score(y_test_classes, y_pred_classes, average='weighted')
    f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-score: {f1:.2f}")

## Performance Evaluation on Malfunctioned Data
In this section, we loop over various combinations of malfunctioning sensors and evaluate the model's performance. This process involves adjusting the data to simulate different malfunction scenarios and then computing key metrics like accuracy, precision, recall, and F1-score for each scenario.


In [ ]:
import itertools

sensors = []
for i in range(1, 6):
    sensors.append(f'Flex-Left-{i}')
    sensors.append(f'Flex-Right-{i}')
sensors.append(f'Position-Left')
sensors.append(f'Position-Right')
sensors.append(f'Orientation-Left')
sensors.append(f'Orientation-Right')

combinations = []
for i in range(1, len(sensors) + 1):
    # Get all combinations of sensors
    combinations += list(itertools.combinations(sensors, i))

print(f"Number of combinations: {len(combinations)}")

In [20]:
# A function to try combinations of sensor values by destroying other sensors (try all combinations)
# Meaning that we will first try to remove 1 sensor, then 2 sensors (with all possible combinations), then 3 sensors, and so on
# The function will return the accuracy, precision, recall, and F1-score for each combination sorted by accuracy
import itertools


def try_combinations(X, y_test):
    # Get the names of the sensors
    sensors = []
    for i in range(1, 6):
        sensors.append(f'Flex-Left-{i}')
        sensors.append(f'Flex-Right-{i}')
    sensors.append(f'Position-Left')
    sensors.append(f'Position-Right')
    sensors.append(f'Orientation-Left')
    sensors.append(f'Orientation-Right')
    
    # Create a list to store the results
    results = []
    # Loop over the sensors
    for i in range(1, len(sensors) + 1):
        # Get all combinations of sensors
        combinations = list(itertools.combinations(sensors, i))
        # Loop over the combinations
        for combination in combinations:
            # Create a copy of the test set
            X_test_copy = X.copy()
            # Destroy the sensors in the combination
            for sensor in combination:
                malfunction(X_test_copy, sensor, 0)
            # Predict the labels
            y_pred = model_rnn.predict(X_test_copy)
            # Convert predictions to classes
            y_pred_classes = np.argmax(y_pred, axis=1)
            y_test_classes = np.argmax(y_test, axis=1)
            # Calculate the accuracy
            accuracy = accuracy_score(y_test_classes, y_pred_classes)
            # Calculate precision, recall, and F1-score
            precision = precision_score(y_test_classes, y_pred_classes, average='weighted')
            recall = recall_score(y_test_classes, y_pred_classes, average='weighted')
            f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')
            # Append the results to the list
            results.append([combination, accuracy, precision, recall, f1])
            print("************************************")
            print(f"Combination {combination}:")
            print(f"Accuracy: {accuracy * 100:.2f}%")
    # Sort the results by accuracy
    results.sort(key=lambda x: x[1], reverse=True)
    return results

In [ ]:
results = try_combinations(X_test, y_test)

## Result Storage and Comprehensive Analysis
The results of our evaluations are stored for detailed analysis. This analysis aims to identify patterns in the model's performance degradation related to specific sensor failures, providing insights into the model's robustness and areas for improvement.


In [22]:
# save the results to a csv file
df_results = pd.DataFrame(results, columns=['Combination', 'Accuracy', 'Precision', 'Recall', 'F1-score'])
df_results.to_csv('results.csv', index=False)

In [ ]:
# Get the combination with accuracy > 0.95 and print its combination and accuracy
max_combination = 0
for result in results:
    if result[1] > 0.95 and len(result[0]) > max_combination:
        max_combination = len(result[0])
        print(f"Combination: {result[0]}")
        print(f"Accuracy: {result[1] * 100:.2f}%")

print(f"Number of sensors: {max_combination}")

# Conclusion
Our investigation into the model's robustness against sensor malfunctions has provided valuable insights. The results indicate how sensor failures impact the overall performance, highlighting the need for fault-tolerant designs in practical applications. Furthermore, through this evaluation, we have gained a clearer understanding of the relative importance of each sensor in our device. We identified certain sensors that do not significantly contribute to the model's predictive ability. This knowledge is invaluable, as it allows us to optimize our design by focusing on the most critical sensors and considering the removal or replacement of sensors that add minimal value. This streamlining could lead to more efficient and cost-effective designs without compromising the functionality and accuracy of our sign language translation tool.